In [1]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

In [3]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

In [4]:
dataset = CustomDataset("../datasets/non_linear.csv")
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])
print(f"Training Data Size : {len(train_dataset)}")
print(f"Validation Data Size : {len(validation_dataset)}")
print(f"Testing Data Size : {len(test_dataset)}")

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

Training Data Size : 160
Validation Data Size : 20
Testing Data Size : 20


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [6]:
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 1000, Model : [Parameter containing:
tensor([[ 3.1069, -1.7020]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.1099], device='cuda:0', requires_grad=True)], Cost : 0.140
Epoch : 2000, Model : [Parameter containing:
tensor([[ 3.1018, -1.7023]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.3357], device='cuda:0', requires_grad=True)], Cost : 0.087
Epoch : 3000, Model : [Parameter containing:
tensor([[ 3.1010, -1.7024]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.4288], device='cuda:0', requires_grad=True)], Cost : 0.077
Epoch : 4000, Model : [Parameter containing:
tensor([[ 3.0989, -1.7027]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.4672], device='cuda:0', requires_grad=True)], Cost : 0.074
Epoch : 5000, Model : [Parameter containing:
tensor([[ 3.0996, -1.7026]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.4832], device='cuda:0', requires_grad=True)]

In [7]:
with torch.no_grad():
    model.eval()
    for x, y in validation_dataloader:
        x = x.to(device)
        y = y.to(device)
        
        outputs = model(x)
        print(f"X : {x}")
        print(f"Y : {y}")
        print(f"Outputs : {outputs}")
        print("--------------------")

X : tensor([[50.4100,  7.1000],
        [64.0000, -8.0000],
        [98.0100,  9.9000],
        [ 0.1600, -0.4000]], device='cuda:0')
Y : tensor([[144.2400],
        [212.6700],
        [287.1300],
        [  1.5500]], device='cuda:0')
Outputs : tensor([[144.6557],
        [212.4865],
        [287.4281],
        [  1.6712]], device='cuda:0')
--------------------
X : tensor([[ 6.2500, -2.5000],
        [59.2900, -7.7000],
        [75.6900,  8.7000],
        [32.4900, -5.7000]], device='cuda:0')
Y : tensor([[ 24.4800],
        [197.8000],
        [220.4600],
        [110.7000]], device='cuda:0')
Outputs : tensor([[ 24.1228],
        [197.3768],
        [220.2887],
        [110.9034]], device='cuda:0')
--------------------
X : tensor([[ 0.0000,  0.0000],
        [39.6900, -6.3000],
        [ 0.0400,  0.2000],
        [ 4.8400,  2.2000]], device='cuda:0')
Y : tensor([[ 1.5000e-01],
        [ 1.3380e+02],
        [-7.0000e-02],
        [ 1.1400e+01]], device='cuda:0')
Outputs : tensor([[  0